In [2]:
import sys
# print(sys.executable)
# print(sys.version)
import cv2
import cvzone

from cvzone.HandTrackingModule import HandDetector

In [3]:
def all_fingers_raised(lmList):

    if len(lmList) != 21:
        return False
        
    thumb_tip = 4
    thumb_ip = 3
    index_tip = 8
    index_ip = 7
    middle_tip = 12
    middle_ip = 11
    ring_tip = 16
    ring_ip = 15
    pinky_tip = 20
    pinky_ip = 19
    return (is_finger_raised(lmList, thumb_tip, thumb_ip) and
            is_finger_raised(lmList, index_tip, index_ip) and
            is_finger_raised(lmList, middle_tip, middle_ip) and
            is_finger_raised(lmList, ring_tip, ring_ip) and
            is_finger_raised(lmList, pinky_tip, pinky_ip))

def is_finger_raised(lmList, tip, phalanx):
    return lmList[tip][1] < lmList[phalanx][1]

In [11]:
hand_detector = HandDetector(detectionCon=0, maxHands=1)
cap = cv2.VideoCapture(0)
cap.set(3, 640)  # Set width
cap.set(4, 480) 

center_x, center_y, width, height = 100, 100, 300, 300


while True:
    rectangle_color = (255, 0, 255)
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    hands, frame = hand_detector.findHands(frame)

    if hands:
        # x and y of tip of your finger

        lmList = hands[0]['lmList']

        index_finger = lmList[8][:2]
        middle_finger = lmList[12][:2]
        distance = ((index_finger[0] - middle_finger[0])**2 + (index_finger[1] - middle_finger[1])**2)**0.5
        # index_middle_distance, _, _ = hand_detector.findDistance(8, 12, frame)

        if all_fingers_raised(lmList):
            rectangle_color = (0, 0, 255)
        
        if distance < 40:
            cursor = lmList[8][:2]
            if center_x - width //2 < cursor[0] < center_x+width //2 and center_y - height //2 < cursor[1] < center_y+height //2:
                rectangle_color = (255, 255, 255)
                center_x, center_y = cursor

    cv2.rectangle(frame, (center_x - width//2, center_y - height//2),
                  (center_x+width//2,center_y+height//2), rectangle_color, cv2.FILLED)
    cv2.imshow("cam", frame)
    
    wait_key = cv2.waitKey(1)
    if wait_key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open zink: /usr/lib/dri/zink_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open zink: /usr/lib/dri/zink_dri.so: cannot open shared object file: 

ModuleNotFoundError: No module named 'pyautogui'